In [1]:
from linefilter import LineFilter
import matplotlib.pyplot as plt
from birdseye import Birdseye
from curves import Curves
from utils import roi
import numpy as np
import pickle
import cv2

In [2]:
calibration_data = pickle.load(open("calibration_data.pkl", "rb"))

camera_matrix = calibration_data['camera_matrix']
dest_coef = calibration_data["distortion_coefficient"]

source_points = [(580, 460), (205, 720), (1110, 720), (703, 460)]
dest_points = [(320, 0), (320, 720), (960, 720), (960, 0)]

p = { 'sat_thresh': 120, 
     'light_thresh': 40, 
     'light_thresh_agr': 205,
     'grad_thresh': (0.7, 1.4), 
     'mag_thresh': 40, 
     'x_thresh': 20 }

birdEye = Birdseye(source_points, dest_points, camera_matrix, dest_coef)
lineFilter = LineFilter(p)
curves = Curves(number_of_windows = 9, margin = 100, minimum_pixels = 50, ym_per_pix = 30 / 720 , xm_per_pix = 3.7 / 700)

In [3]:
def curves_test(path):
    img = cv2.imread(path)
    binary = lineFilter.apply(img)
    wb = np.logical_and(birdEye.sky_view(binary), roi(binary)).astype(np.uint8)
    result = curves.fit(wb)
    print("[real world] left best-fit curve parameters:", result['real_left_best_fit_curve'])
    print("[real world] right best-fit curve parameters:", result['real_right_best_fit_curve'])
    print("[pixel] left best-fit curve parameters:", result['pixel_left_best_fit_curve'])
    print("[pixel] left best-fit curve parameters:", result['pixel_right_best_fit_curve'])
    print("[left] current radius of curvature:", result['left_radius'], "m")
    print("[right] current radius of curvature:", result['right_radius'], "m")
    print("vehicle position:", result['vehicle_position_words'])
    ground_img_with_projection = birdEye.project(img, binary, result['pixel_left_best_fit_curve'], result['pixel_right_best_fit_curve'])
    plt.imshow(result['image'])
    plt.imshow(ground_img_with_projection)

In [4]:
curves_test("test_images/test1.jpg")

[real world] left best-fit curve parameters: [ 4.39918667e-04 -2.51161790e-02  2.18532172e+00]
[real world] right best-fit curve parameters: [ 2.44421587e-04 -2.51085032e-02  5.73192779e+00]
[pixel] left best-fit curve parameters: [ 1.44492805e-04 -1.97987897e-01  4.13439244e+02]
[pixel] left best-fit curve parameters: [ 8.02811144e-05 -1.97927390e-01  1.08441877e+03]
[left] current radius of curvature: 1136.576518419726 m
[right] current radius of curvature: 2045.9804867443568 m
vehicle position: 0.13 m right of center


AttributeError: 'Birdseye' object has no attribute 'inv_warp_matrix'

In [ ]:
ground_img_with_projection = birdsEye.project(ground_img, binary, left_fit_curve, right_fit_curve)